## Discovering Confounding in multiple Contexts (Chapter 4)

In [1]:
%load_ext autoreload
%autoreload 2
import os, sys

import numpy as np

from src.coco.gen.dag_confounded import DAGConfounded
from src.coco.gen.dag_gen import _random_nonlinearity
from src.coco.util.mi_sampling import Sampler
from src.coco.coco import CoCo

sys.path.insert(0, os.path.abspath(".."))

def _show(res, method, id):
    tp, fp, tn, fn, f1, _, _ = res
    print(f'\t{method}-{id}: \t(f1={np.round(f1, 2)})'
          f'\t(tp={tp}, tn={tn}, fp={fp}, fn={fn})')



C:\Users\msara\anaconda3\envs\causalchangetest\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

# parameters to test
n_nodes, n_confounders, n_contexts, n_samples = (5, 1, 5, 500)
n_shifts_observed, n_shifts_confounders = 1, 1
fun_form = _random_nonlinearity()

# hyperparameters
seed = 42
FCI_INDEP_TEST = 'kci' # 'fisherz'

# Data Generation
sampler = Sampler()
dag = DAGConfounded(seed, n_contexts, n_nodes, n_confounders, n_shifts_observed, n_shifts_confounders,
                    is_bivariate=False)

D, Dobs = dag.gen_data(seed, n_samples, _functional_form=fun_form, oracle_partition=True, noise_iv=False)


# with known number of confounders
coco_oZ = CoCo(D, dag.G.nodes, sampler=sampler, n_components=n_confounders, dag=dag)
_show(coco_oZ.eval_estimated_edges(dag), coco_oZ, 'cfd')

# with unknown number of confounders
coco = CoCo(D, dag.G.nodes, sampler=sampler, n_components=None, dag=dag, verbosity=0)
_show(coco.eval_estimated_edges(dag), coco, 'cfd')

res = coco.eval_causal(dag)
res = res[0], res[1], res[2], res[3], res[4], 0, 0

_show(res, coco, 'caus')


from src.coco.util.co_test_types import CoShiftTestType, CoDAGType

# CoCo with known causal directions
coco_oG = CoCo(D, dag.G.nodes, sampler=sampler, n_components=None,
               dag_discovery=CoDAGType.SKIP, dag=dag)
_show(coco_oG.eval_estimated_edges(dag), coco_oG, 'cfd')

# CoCo with known causal directions as well as partitions for each node
coco_oPi = CoCo(D, dag.G.nodes, sampler=sampler, n_components=None,
                shift_test=CoShiftTestType.SKIP,
                dag_discovery=CoDAGType.SKIP, dag=dag)

_show(coco_oPi.eval_estimated_edges(dag), coco_oPi, 'cfd')

from src.exp.exp_coco.method_types import MethodType
from src.exp.exp_coco.run_coco import run_fci

fci_jci, D, Dobs, seed = run_fci(dag, D, Dobs, n_samples, fun_form, MethodType.FCI_JCI, FCI_INDEP_TEST, seed)

tp, fp, tn, fn, f1 = fci_jci.eval_confounded(dag, MethodType.FCI_JCI)
print(f'\tJFCI-cfd: \t(f1={np.round(f1, 2)})\t(tp={tp}, tn={tn}, fp={fp}, fn={fn})')

tp, fp, tn, fn, f1 = fci_jci.eval_causal(dag, MethodType.FCI_JCI)
print(f'\tJFCI-caus: \t(f1={np.round(f1, 2)})\t(tp={tp}, tn={tn}, fp={fp}, fn={fn})')



	coco_oZ-cfd: 	(f1=0.29)	(tp=1, tn=4, fp=0, fn=5)
	coco-cfd: 	(f1=0.29)	(tp=1, tn=4, fp=0, fn=5)
	coco-caus: 	(f1=0.6)	(tp=3, tn=13, fp=2, fn=2)
	coco_oG-cfd: 	(f1=0.67)	(tp=3, tn=4, fp=0, fn=3)
	coco_oG_oPi-cfd: 	(f1=0.0)	(tp=0, tn=4, fp=0, fn=6)
NODES: ['X6', 'X3', 'X5', 'X4', 'X7'] [np.int64(6), np.int64(3), np.int64(5), np.int64(4), np.int64(7)]
NODES: ['X6', 'X3', 'X5', 'X4', 'X7', 'X8', 'X9', 'X10', 'X11'] 	 [np.int64(6), np.int64(3), np.int64(5), np.int64(4), np.int64(7)] ['C0', 'C1', 'C2', 'C3']
ADJ:
[[0. 1. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0.]]
CPDAG:
[[0 1 1 0 0]
 [1 0 0 1 0]
 [1 0 0 0 1]
 [0 1 0 0 1]
 [0 0 0 0 0]]
X5 -> X7
X4 -> X7
context 7
X10 o X11
X11 o X10
X6 o X10
X3 o X10
X5 o X10
X4 o X10
X7 o X10
context 8
X6 o X11
X3 o X11
X5 o X11
X4 o X11
X7 o X11
NODES: ['X6', 'X3', 'X5', 'X4', 'X7'] [np.int64(6), np.int64(3), np.int64(5), np.int64(4), np.int64(7)]
ADJ:
[[0. 1. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 

Depth=0, working on node 8: 100%|██████████| 9/9 [00:11<00:00,  1.23s/it]
